In [1]:
## Imports

import os

print(os.cpu_count())

import copy
import wandb

wandb.require("core")

import pandas as pd
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns

import torch
from torchvision.io import read_image
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
from torchvision import transforms
import torch.optim as optim
from torch import nn
import torch.backends.cudnn as cudnn
from torcheval.metrics.functional import binary_auroc

from torchvision.models import (
    convnext_tiny,
    convnext_small,
)

from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedGroupKFold
from sklearn.metrics import roc_curve, auc, roc_auc_score
from sklearn.preprocessing import MinMaxScaler

import albumentations as A
from albumentations.pytorch import ToTensorV2

from colorama import Fore, Style
b_ = Fore.BLUE
sr_ = Style.RESET_ALL

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps" if torch.backends.mps.is_available() else "cpu"
)
print(f"Using {device} device")

cudnn.benchmark = True
torch.set_float32_matmul_precision("high")

30


INFO:albumentations.check_version:A new version of Albumentations is available: 1.4.12 (you have 1.4.11). Upgrade using: pip install --upgrade albumentations


Using cuda device


In [2]:
configs = dict(
    image_size = 224,
)

In [3]:
train_metadata_df = pd.read_csv("../data/stratified_5_fold_train_metadata.csv")

def add_path(row):
    return f"../data/train-image/image/{row.isic_id}.jpg"

train_metadata_df["path"] = train_metadata_df.apply(lambda row: add_path(row), axis=1)

cols = ["tbp_lv_symm_2axis", "tbp_lv_areaMM2", "tbp_lv_color_std_mean", "tbp_lv_norm_border", "tbp_lv_perimeterMM"]
train_metadata_df = train_metadata_df[["path", "fold"]+cols]
train_metadata_df.head()

/tmp/ipykernel_1361848/1675229643.py:1: DtypeWarning: Columns (51,52) have mixed types. Specify dtype option on import or set low_memory=False.
  train_metadata_df = pd.read_csv("../data/stratified_5_fold_train_metadata.csv")


,path,fold,tbp_lv_symm_2axis,tbp_lv_areaMM2,tbp_lv_color_std_mean,tbp_lv_norm_border,tbp_lv_perimeterMM
0,../data/train-image/image/ISIC_0015670.jpg,4,0.590476,3.152561,0.00000,7.091360,9.307003
1,../data/train-image/image/ISIC_0015845.jpg,2,0.285714,0.919497,0.00000,2.116402,3.354148
2,../data/train-image/image/ISIC_0015864.jpg,4,0.361905,3.265153,0.00000,4.798335,8.886309
3,../data/train-image/image/ISIC_0015902.jpg,1,0.209581,6.079940,0.51452,1.975874,9.514499
4,../data/train-image/image/ISIC_0024200.jpg,4,0.313433,2.101708,0.00000,3.658854,6.467562


In [4]:
scaler = MinMaxScaler()

train_metadata_df[cols] = scaler.fit_transform(train_metadata_df[cols])
train_metadata_df.head()

,path,fold,tbp_lv_symm_2axis,tbp_lv_areaMM2,tbp_lv_color_std_mean,tbp_lv_norm_border,tbp_lv_perimeterMM
0,../data/train-image/image/ISIC_0015670.jpg,4,0.582086,0.008153,0.000000,0.690918,0.067335
1,../data/train-image/image/ISIC_0015845.jpg,2,0.252621,0.001462,0.000000,0.162262,0.007756
2,../data/train-image/image/ISIC_0015864.jpg,4,0.334987,0.008491,0.000000,0.447253,0.063125
3,../data/train-image/image/ISIC_0015902.jpg,1,0.170317,0.016925,0.051695,0.147329,0.069412
4,../data/train-image/image/ISIC_0024200.jpg,4,0.282586,0.005005,0.000000,0.326168,0.038916


## Dataloader

In [5]:
class PretrainSkinDataset(Dataset):
    def __init__(self, df: pd.DataFrame, transform=None):
        assert "path" in df.columns
        assert "tbp_lv_symm_2axis" in df.columns
        # TODO: add more features

        self.paths = df.path.tolist()
        self.labels = df[cols].values # continuous, float
        self.transform = transform

    def __len__(self):
        return len(self.paths)

    def __getitem__(self, idx: int):
        image = read_image(self.paths[idx]).to(torch.float32) / 255.0
        label = self.labels[idx]
        if self.transform:
            image = image.numpy().transpose((1,2,0))
            image = self.transform(image=image)["image"]
        return image, label

In [6]:
train_df = train_metadata_df.loc[
    train_metadata_df.fold != 0
]
valid_df = train_metadata_df.loc[
    train_metadata_df.fold == 0
]

In [7]:
# # placeholders
# psum = torch.tensor([0.0, 0.0, 0.0])
# psum_sq = torch.tensor([0.0, 0.0, 0.0])

# num_workers = 24 # based on profiling

# simple_transform = A.Compose([
#     A.Resize(configs["image_size"], configs["image_size"]),
#     ToTensorV2(),
# ])

# _train_dataset = PretrainSkinDataset(train_df, transform=simple_transform)
# _train_dataloader = DataLoader(
#     _train_dataset, batch_size=128, shuffle=True,
#     num_workers=num_workers, pin_memory=True, persistent_workers=True
# )

# # loop through images
# for inputs, labels in tqdm(_train_dataloader):
#     psum += inputs.sum(axis=[0, 2, 3])
#     psum_sq += (inputs**2).sum(axis=[0, 2, 3])

# count = len(train_df) * configs["image_size"] * configs["image_size"]
# total_mean = psum / count
# total_var = (psum_sq / count) - (total_mean**2)
# total_std = torch.sqrt(total_var)
# print("mean: " + str(total_mean))
# print("std:  " + str(total_std))

In [8]:
transforms_train = A.Compose([
    A.Transpose(p=0.5),
    A.VerticalFlip(p=0.5),
    A.HorizontalFlip(p=0.5),
    A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, p=0.75),
    A.OneOf([
        A.MotionBlur(blur_limit=5),
        A.MedianBlur(blur_limit=5),
        A.GaussianBlur(blur_limit=5),
        A.GaussNoise(var_limit=(5.0, 30.0)),
    ], p=0.7),

    A.OneOf([
        A.OpticalDistortion(distort_limit=1.0),
        A.GridDistortion(num_steps=5, distort_limit=1.),
        A.ElasticTransform(alpha=3),
    ], p=0.7),
    A.HueSaturationValue(hue_shift_limit=10, sat_shift_limit=20, val_shift_limit=10, p=0.5),
    A.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.1, rotate_limit=15, border_mode=0, p=0.85),
    A.Resize(configs["image_size"], configs["image_size"]),
    A.Normalize(
        mean=(0.6962, 0.5209, 0.4193),
        std=(0.1395, 0.1320, 0.1240)
    ),
    ToTensorV2(),
])

transforms_val = A.Compose([
    A.Resize(configs["image_size"], configs["image_size"]),
    A.Normalize(
        mean=(0.6962, 0.5209, 0.4193),
        std=(0.1395, 0.1320, 0.1240)
    ),
    ToTensorV2(),
])

/home/ubuntu/.local/lib/python3.10/site-packages/pydantic/main.py:164: UserWarning: blur_limit and sigma_limit minimum value can not be both equal to 0. blur_limit minimum value changed to 3.
  __pydantic_self__.__pydantic_validator__.validate_python(data, self_instance=__pydantic_self__)
/home/ubuntu/.local/lib/python3.10/site-packages/pydantic/main.py:308: UserWarning: Pydantic serializer warnings:
  Expected `Union[float, json-or-python[json=list[float], python=list[float]]]` but got `tuple` - serialized value may not be as expected
  Expected `Union[float, json-or-python[json=list[float], python=list[float]]]` but got `tuple` - serialized value may not be as expected
  return self.__pydantic_serializer__.to_python(


In [9]:
# # Calculate statistics
# mean_train = train_df['tbp_lv_symm_2axis'].mean()
# median_train = train_df['tbp_lv_symm_2axis'].median()
# mean_valid = valid_df['tbp_lv_symm_2axis'].mean()
# median_valid = valid_df['tbp_lv_symm_2axis'].median()

# # Plot the distributions
# plt.figure(figsize=(10, 6))
# sns.histplot(train_df['tbp_lv_symm_2axis'], color='blue', label='Train Set', kde=True, alpha=0.5)
# sns.histplot(valid_df['tbp_lv_symm_2axis'], color='green', label='Validation Set', kde=True, alpha=0.5)

# # Add mean and median lines and annotations
# plt.axvline(mean_train, color='blue', linestyle='dashed', linewidth=1)
# plt.axvline(median_train, color='blue', linestyle='solid', linewidth=1)
# plt.axvline(mean_valid, color='green', linestyle='dashed', linewidth=1)
# plt.axvline(median_valid, color='green', linestyle='solid', linewidth=1)

# plt.text(mean_train, plt.ylim()[1]*0.8, f'Mean: {mean_train:.2f}', color='blue', ha='center')
# plt.text(median_train, plt.ylim()[1]*0.7, f'Median: {median_train:.2f}', color='blue', ha='center')
# plt.text(mean_valid, plt.ylim()[1]*0.6, f'Mean: {mean_valid:.2f}', color='green', ha='center')
# plt.text(median_valid, plt.ylim()[1]*0.5, f'Median: {median_valid:.2f}', color='green', ha='center')

# # Add title and labels
# plt.title('Distribution of tbp_lv_symm_2axis in Train and Validation Sets')
# plt.xlabel('tbp_lv_symm_2axis')
# plt.ylabel('Density')
# plt.legend()

# # Show plot
# plt.show()

In [10]:
num_workers = 24 # based on profiling

train_dataset = PretrainSkinDataset(train_df, transform=transforms_train)
train_dataloader = DataLoader(train_dataset, batch_size=128, shuffle=True, num_workers=num_workers, pin_memory=True, persistent_workers=True, drop_last=True)

valid_dataset = PretrainSkinDataset(valid_df, transform=transforms_val)
valid_dataloader = DataLoader(valid_dataset, batch_size=128, shuffle=False, num_workers=num_workers, pin_memory=True, persistent_workers=True, drop_last=True)

dataset_sizes = {"train": len(train_dataset), "val": len(valid_dataset)}
dataset_sizes

{'train': 320848, 'val': 80211}

In [11]:
imgs, labels = next(iter(train_dataloader))

## Model

In [12]:
model_ft = convnext_tiny()
model_ft.classifier[2] = nn.Linear(model_ft.classifier[2].in_features, 5, bias=False)

model_ft = model_ft.to(device)
model_ft = torch.compile(model_ft)
print(model_ft)

OptimizedModule(
  (_orig_mod): ConvNeXt(
    (features): Sequential(
      (0): Conv2dNormActivation(
        (0): Conv2d(3, 96, kernel_size=(4, 4), stride=(4, 4))
        (1): LayerNorm2d((96,), eps=1e-06, elementwise_affine=True)
      )
      (1): Sequential(
        (0): CNBlock(
          (block): Sequential(
            (0): Conv2d(96, 96, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), groups=96)
            (1): Permute()
            (2): LayerNorm((96,), eps=1e-06, elementwise_affine=True)
            (3): Linear(in_features=96, out_features=384, bias=True)
            (4): GELU(approximate='none')
            (5): Linear(in_features=384, out_features=96, bias=True)
            (6): Permute()
          )
          (stochastic_depth): StochasticDepth(p=0.0, mode=row)
        )
        (1): CNBlock(
          (block): Sequential(
            (0): Conv2d(96, 96, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), groups=96)
            (1): Permute()
            (2): LayerNorm

## Utils

In [16]:
def train_model(model, dataloader, criterions, optimizer, scheduler=None):
    model.train()  # Set model to training mode

    running_loss = 0.0

    for idx, (inputs, labels) in enumerate(dataloader):
        inputs = inputs.to(device)
        labels = labels.to(device).flatten().to(torch.float32)
        print(f"{labels.shape=}")

        optimizer.zero_grad()

        with torch.set_grad_enabled(True):
            outputs = model(inputs).flatten()
            print(f"{outputs.shape=}")
            _losses = []
            for idx, criterion in enumerate(criterions):
                _losses.append(criterion(outputs[:, idx], labels[:, idx]))

            loss = sum(_losses)
            loss.backward()
            optimizer.step()

        loss_val = loss.detach()
        running_loss += loss_val * inputs.size(0)

        if (idx + 1) % 10 == 0:
            wandb.log({"train_loss": loss_val})
            print(f"Train Batch Loss: {loss_val}")

    epoch_loss = running_loss / dataset_sizes["train"]

    return model, epoch_loss


def validate_model(model, dataloader, criterions, optimizer):
    model.eval()

    running_loss = 0.0

    for idx, (inputs, labels) in enumerate(dataloader):
        inputs = inputs.to(device)
        labels = labels.to(device).flatten().to(torch.float32)

        optimizer.zero_grad()

        with torch.no_grad():
            outputs = model(inputs).flatten()
            _losses = []
            for idx, criterion in criterion(criterions):
                _losses.append(criterion(outputs[:, idx], labels[:, idx]))
        
        loss = sum(_losses)
        loss_val = loss.detach()
        running_loss += loss_val * inputs.size(0)

        if (idx + 1) % 10 == 0:
            wandb.log({"valid_loss": loss_val})
            print(f"valid Batch Loss: {loss_val}")

    valid_loss = running_loss / dataset_sizes["val"]

    return model, valid_loss

## Pretrain

In [17]:
criterions = [nn.MSELoss()]*5
optimizer_ft = optim.Adam(model_ft.parameters(), lr=0.001, weight_decay=1e-6)

In [18]:
run = wandb.init(project="isic_lesions_24", job_type="pretrain")

best_epoch_auroc = -np.inf
best_valid_loss = np.inf
early_stopping_patience = 4
epochs_no_improve = 0

for epoch in range(10):
    model_ft, epoch_loss = train_model(
        model_ft, train_dataloader, criterions, optimizer_ft
    )
    model_ft, valid_loss = validate_model(
        model_ft, valid_dataloader, criterions, optimizer_ft
    )

    print(
        f"Epoch: {epoch} | Train Loss: {epoch_loss} | Valid Loss: {valid_loss}\n"
    )
    wandb.log(
        {
            "epoch": epoch,
            "epoch_loss": epoch_loss,
            "epoch_val_loss": valid_loss,
        }
    )

    # earlystopping dependent on validation loss
    if best_valid_loss >= valid_loss:
        print(f"{b_}Validation Loss Improved ({best_valid_loss} ---> {valid_loss}){sr_}")
        
        # checkpointing
        best_model_wts = copy.deepcopy(model_ft.state_dict())
        PATH = "../models/pretrain_valid_loss{:.4f}_epoch{:.0f}.bin".format(valid_loss, epoch)
        torch.save(model_ft.state_dict(), PATH)
        # Save a model file from the current directory
        print(f"{b_}Model Saved{sr_}")
        best_valid_loss = valid_loss
        epochs_no_improve = 0
    else:
        epochs_no_improve += 1

    if epochs_no_improve >= early_stopping_patience:
        print(
            f"{b_}Early stopping triggered after {epochs_no_improve} epochs with no improvement.{sr_}"
        )
        break

labels.shape=torch.Size([640])


OutOfMemoryError: CUDA out of memory. Tried to allocate 74.00 MiB. GPU 